In [1]:
import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import statistics

In [2]:
def readTimeTrace(fdir, nRanks) :
    TIMES = []
    EVENTS = []
    CNT = 0
    print('dir=', fdir, 'nRanks=', nRanks)
    for r in range(nRanks) :
        fname = '%s/timetrace.%d.out'% (fdir, r)
        #print('    reading: ', fname)
        f = open(fname, 'r')
        allLines = f.readlines()
        #if r == 1 : print(allLines)

        TIME = []
        ADV = []
        COMM = []
        T0 = -1
        AT0, AT1 = (0,0)
        C0, C1 = (0,0)
        EVENT = []
        for line in allLines :
            x = line.strip().split(' ')
            event = x[0].split('_')[0]
            step = int(x[0].split('_')[1])
            time = float(x[1])
            if step == 1 : continue
            
            #print('x=', x, 'step=',step, 'event=', event)
            if len(TIME) == 0 and event == 'GoStart': T0 = time
            time = time - T0
            if event == 'AdvectStart' : AT0 = time
            if event == 'AdvectEnd' :
                AT1 = time
                if AT1-AT0 > 1 : 
                    TIME.append(AT0)
                    TIME.append(AT1)
                    EVENT.append(['A', 'A'])
            if event == 'CommStart' : C0 = time
            if event == 'SendDataEnd' : 
                C1 = time
                if C1-C0 > 1 :
                    TIME.append(C0)
                    TIME.append(C1)
                    EVENT.append(['C', 'C'])
                    #print('************ COMM = ', (C0,C1))

            #elif event == 'CommStart' :   TIME.append(time)
            #elif event == 'SyncCommStart' : TIME.append(time)
        TIMES.append(TIME)
        EVENTS.append(EVENT)
    return TIMES, EVENTS

def getTimeRange(TIMES) :
    tMin = 1e20
    tMax = -1e20

    for t in TIMES[1:] :
        if len(t) > 1 :
            tMin = min(tMin, t[0])
            tMax = max(tMax, t[-1])
    return (tMin, tMax)

def participationBins(TIMES, numBins) :
    (tMin, tMax) = getTimeRange(TIMES)
    dT = tMax/(numBins)
    #print('************ tMin/Max= ', (tMin, tMax), 'dT= ', dT)

    ALLPT = []
    CNT = 0
    for TIME in TIMES :
        PT = [0] * numBins
        N = len(TIME)
        idx = 0
        while idx < N :
            binIdx0 = int(TIME[idx] / dT)
            binIdx1 = int(TIME[idx+1] / dT)
            if binIdx0 >= numBins : binIdx0 = numBins-1
            if binIdx1 >= numBins : binIdx1 = numBins-1
            #if CNT == 0 : print(idx, ': ******** bins: ', binIdx0, binIdx1, TIME[idx], TIME[idx+1])
            PT[binIdx0] = 1
            for b in range(binIdx0, binIdx1) : PT[b] = 1
            idx = idx+2
        ALLPT.append(PT)
        CNT = CNT + 1
    return ALLPT

def calcParticipation(ALLBINS, nRanks, numBins) :
    PARTICIPATION = [0]*numBins
    for b in range(numBins) :
        for r in range(nRanks) :
            PARTICIPATION[b] += ALLBINS[r][b]
        PARTICIPATION[b] = float(PARTICIPATION[b]) / float(nRanks)
    return PARTICIPATION


def mkLabel(imageNm) :
    res = imageNm
    res = imageNm.replace('0.clover', '0')
    res = res.replace('.B.', '.WHOLE.')
    res = res.replace('.B0.', '.BOX50.')
    res = res.replace('.B1.', '.BOX25.') 
    res = res.replace('.B2.', '.BOX10.')
    res = res.replace('.B3.', '.BOX05.')

    return res

def plotParticipation(pdata, TIMES, numBins, imageNm) :
    (tMin, tMax) = getTimeRange(TIMES)
    dT = tMax/(numBins)
    #print('************************************* dT= ', dT)
    X = [0]
    for b in range(numBins-1) :
        X.append(X[-1] + dT)
    SUM = sum(pdata)
    
    #print('X=', X, len(X))
    #print('Y=', pdata, len(pdata))
    
    imgLabel = mkLabel(imageNm)
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.title.set_text('%s Avg= %f' %(imgLabel, SUM/numBins))
    ax.set_ylim([0.0, 1.1])
    ax.set_xlabel('Time (ms)', fontsize="large")
    ax.set_aspect('auto')
    ax.set_ylabel('Rank Participation', fontsize="large")
    ax.plot(X, pdata)
    #fig.savefig(imageNm + '.png')
    
    

In [3]:
cd /gpfs/alpine/csc143/proj-shared/pugmire/forJay/VisPerfExp/ubuntu_cpu_vtkm2.0/runDIR/weak-weak-output/

/gpfs/alpine/csc143/proj-shared/pugmire/forJay/VisPerfExp/ubuntu_cpu_vtkm2.0/runDIR/weak-weak-output


In [4]:
def getRuns(fSel=None, blockSel=None, nodeSel=None, boxSel=None, parSel=None) :
    allFiles = os.listdir('.')
    res = []
    for f in allFiles :
        x = f.split('.')

        if len(x) != 4 : continue
        y = x[3].split('_')
        takeIt = True
        #print(x, 'B' in x[3])
        if fSel and x[0] not in fSel : takeIt = False
        if blockSel and x[1] not in blockSel : takeIt = False
        if nodeSel and x[2] not in nodeSel : takeIt = False
        if parSel and y[1] not in parSel : takeIt = False
        if boxSel and takeIt :
            if type(boxSel) == str : takeIt = boxSel == y[0]
            else : takeIt = y[0] in boxSel
     
        if takeIt : res.append(f)
    return res

In [5]:
BLOCK_MAP = {}
BLOCK_MAP[32] =  '2_4_4'
BLOCK_MAP[64] =  '4_4_4'
BLOCK_MAP[128] = '4_4_8'
BLOCK_MAP[256] = '4_8_8'
BLOCK_MAP[512] = '8_8_8'

In [13]:
NBLOCKS=256
NUM_RANKS = NBLOCKS

runs = getRuns(fSel=['clover'], blockSel=BLOCK_MAP[NBLOCKS], boxSel='B')
runs = getRuns(fSel=['clover', 'astro', 'fusion', 'fishtank'], nodeSel='n16', blockSel=BLOCK_MAP[NBLOCKS], boxSel='B')

def cmp (x) :
    v = int(x.split('.')[2][1:])
    #print('x=',v)
    return v

if len(runs) > 0 : 
    runs = sorted(runs, key=cmp)
runs.sort()
print(runs)

['clover.4_8_8.n16.B_p10000_s1000']


In [14]:
NUM_BINS = 30
print(len(runs))

for run in runs :
    #print('reading: ', run)
    TIMES, EVENTS = readTimeTrace(run, NUM_RANKS)
    #print('T0= ', TIMES[0])
    ALLBINS = participationBins(TIMES, NUM_BINS)
    #print('ALLBINS[0]=', ALLBINS[0])
    #print('***', TIMES[0])
    #print('ALLBINS[1]=', ALLBINS[1])
    #print('***', TIMES[1])

    #print('ALLBINS[31]=', ALLBINS[1])

    PARTICIPATION = calcParticipation(ALLBINS, NUM_RANKS, NUM_BINS)
    plotParticipation(PARTICIPATION, TIMES, NUM_BINS, run)
    #print(PARTICIPATION)


1
dir= clover.4_8_8.n16.B_p10000_s1000 nRanks= 256


FileNotFoundError: [Errno 2] No such file or directory: 'clover.4_8_8.n16.B_p10000_s1000/timetrace.0.out'

/gpfs/alpine/csc143/proj-shared/pugmire/forJay/VisPerfExp/ubuntu_cpu_vtkm2.0/runDIR/weak-weak-output


In [62]:
cd /gpfs/alpine/csc143/proj-shared/pugmire/forJay/VisPerfExp/ubuntu_cpu_vtkm2.0/runDIR

/gpfs/alpine/csc143/proj-shared/pugmire/forJay/VisPerfExp/ubuntu_cpu_vtkm2.0/runDIR


In [70]:
###poo poo

NUM_RANKS=8
NUM_BINS=220

TIMES, EVENTS = readTimeTrace('.', 8)

ALLBINS = participationBins(TIMES, NUM_BINS)
m,M = (0,0)
for i in range(len(TIMES)) :
    print('rank:', i, (TIMES[i][0], TIMES[i][-1]), 'N=', len(TIMES[i]), '\n', ALLBINS[i])
    if TIMES[i][-1] > M : M = TIMES[i][-1]
    
print(TIMES[2])
print(EVENTS[2])
dX = float(M) / float(NUM_BINS-1)
X = []
x = 0
for i in range(NUM_BINS) :
    X.append(i* dX)

#PARTICIPATION = calcParticipation(ALLBINS, NUM_RANKS, NUM_BINS)
#print(PARTICIPATION)
#plotParticipation(PARTICIPATION, TIMES, NUM_BINS, 'test')

#X = list(range(NUM_BINS))
print(len(X))
print(len(ALLBINS[0]))

fig, ax = plt.subplots(figsize=(8,4))

Y = [0] * NUM_BINS
for r in range(NUM_RANKS) :
    for b in range(NUM_BINS) :
        Y[b] = Y[b] + ALLBINS[r][b]
print('************************** Y ************************')
print(Y)
print(X)
ax.plot(X, Y)


NameError: name 'shit' is not defined